In [ ]:
import pandas as pd
from io import StringIO
import re
import os
df = pd.read_csv("Case_Chap3_Sec2_Consolidated/temp.csv")
df.shape

In [2]:
def get_dates(list_num):
    dates = []
    for i in range(len(list_num)):
        try:
            year, month, day = list_num[i:i+3]
            assert 2000 <= year <= 2025
            assert 1 <= month <= 12
            assert 1 <= day <= 31
            dates.append(f"{year}.{month:02d}.{day:02d}")
        except: pass

    return dates

In [3]:
list_name = []
for iii in range(len(df)):
    path = df.iloc[iii]["path"]
    lines = df.iloc[iii]["lines"].splitlines()
    table = df.iloc[iii]["table"]
    temp_df = pd.read_csv(StringIO(table))
    if temp_df.shape[1]+2 != len(lines):
        assert temp_df.shape[1]+1 == len(lines)
        list_name.append(lines[0])
        start_index = 1
    else:
        list_name.append(lines[1])
        start_index = 2

    if path.endswith("BS.html"):
        assert list_name[-1].endswith("재무상태표")
    elif path.endswith("CIC.html"):
        assert list_name[-1].endswith("포괄손익계산서")
    elif path.endswith("IC.html"):
        assert list_name[-1].endswith("손익계산서")
    else: raise

    currency = re.findall(r"\(단위\:(.*?)\)", lines[-1])
    assert len(currency) == 1
    currency = currency[0]

    list_date = []
    for line in lines[start_index:-1]:
        list_num = [int(_) for _ in re.findall(r"\d+", line)]
        dates = get_dates(list_num)
        if path.endswith("BS.html"):
            assert len(dates) == 1
        elif path.endswith("IC.html"):
            assert len(dates) == 2
        else: raise

        if len(dates) == 1:
            list_date.append(dates[0])
        else:
            list_date.append("~".join(dates))

    assert temp_df.columns[0] == "Unnamed: 0" and len(temp_df.columns) - 1 == len(list_date)
    temp_df.columns = ["Accounts"] + list_date

    new_path = path.replace("Html_subsec", "Raw_tables").replace(".html", f"_{currency}.csv")
    os.makedirs(os.path.dirname(new_path), exist_ok=True)
    temp_df.to_csv(new_path, index=False)